### Underfitting and Overfitting
### One-hot and Embedding
### Normalize and Scale
### Machine Learning pipeline


### 常用机器学习模型
##### 1. KNN
##### 2. Decision Tree
##### 3. Support vector machine
##### 4. Bayes classifier
##### 5. Boosting

### KNN的优点和缺点
##### 优点
- 容易实现
- 模型容易调整, 可以方便的获取K的数量, 或者给不同距离的K进行加权
- 适合解决各种复杂问题 (分类, 回归, 高维, 低维, 复杂关系, 简单关系)

##### 缺点
- KNN运行时间久
- 容易被异常值影响
- 所需空间大
- 高维空间的距离区分度不大



### Decision Tree
##### 优点
- 1. clear to explain
- 2. could select salient features

##### 缺点
- 1. so sensitive
- 2. fit ability is limited

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier, export_graphviz

In [4]:
iris_data = load_iris()
x_data = iris_data.data
y_data = iris_data.target

In [5]:
tree_clf = DecisionTreeClassifier()
tree_clf.fit(x_data, y_data)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [6]:
export_graphviz(tree_clf, 
                out_file='iris_tree.dot', 
                feature_names=iris_data.feature_names, 
                class_names=iris_data.target_names, 
                rounded=True, 
                filled=True)

In [12]:
!dot -Tpdf iris_tree.dot -o outfile.pdf